# Filmes e Séries: Netflix

## Parte 1: Coleta e Análise Primária dos Dados

In [1]:
# Importando as bibliotecas que serão utilizadas para a realização dos trabalhos:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import re

# Definindo algumas configurações iniciais em relação a estilização e dimensões de gráficos 
# para plotagem posterior:
sns.set(rc={'figure.figsize':(11.7, 8.27)})
sns.set(style='darkgrid')